<!-- vscode-jupyter-toc -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
<a id='toc0_'></a>**Содержание**    
- [Общий алгоритм работы с текстами с помощью нейросетей](#toc1_)    
- [Дистрибутивная семантика и векторные представления слов](#toc2_)    
    - [Дистрибутивная гипотеза](#toc2_1_1_)    
    - [Общий алгоритм](#toc2_1_2_)    
      - [Сглаживание](#toc2_1_2_1_)    
      - [Разложение](#toc2_1_2_2_)    
    - [Модель GloVe (глобальные вектора)](#toc2_1_3_)    
    - [Модель word2vec](#toc2_1_4_)    
      - [Рассмотрим более подробно SkipGram](#toc2_1_4_1_)    
      - [Negative Sampling](#toc2_1_4_2_)    
    - [Модель fastText](#toc2_1_5_)    
      - [Общий алгоритм обучения FastText Skip Gram Negative Sampling выглядит следующим образом:](#toc2_1_5_1_)    
- [Теоретические вопросы: Дистрибутивная семантика](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- /vscode-jupyter-toc -->

# <a id='toc1_'></a>[Общий алгоритм работы с текстами с помощью нейросетей](#toc0_)

Общая схема **предобработки текста** (перевода текста в векторное пространство):

1. очищаем текст от мусора (разметки, оформления) — того, что считаем нерелевантным в нашей задаче

2. разбиваем текст на базовые элементы (символы или токены в зависимости от задачи)

    -> список объектов, очищенных и нормализованных
    
3. пересчитываем все уникальные элементы в корпусе и сопоставляем каждому элементу некоторый идентификатор
    
4. получаем отображение из символов или токенов в числа и применяем его к предобработанным текстам

    -> получим такие же списки, в которых каждый элемент заменён на его глобальный идентификатор
    $$ \text {Text} \longrightarrow \text {Matrix}_{length \times embedding\_size}$$

`мама 0,1 ... 0.5`  
`...`  
`раму 0,2 ... 0,1`  

На этом предобработка текста для нейросетей заканчивается и начинаются сами нейросети.


5. преобразование тензора текста с целью получения информации о локальном контексте
$$ \text {Text} \longrightarrow \text {Matrix}_{length \times embedding\_size} \longrightarrow \text {Matrix}_{NEW\_length \times NEW\_embedding\_size}$$

6. агрегировать тензор для принятия решений с целью получения объекта фиксированной размерности под требования конкретной задачи (дальше с таким объектом (обычно вектором) можно классифицировать, конкатенировать с другими, и так далее)

$$ \text {Text} \longrightarrow \text {Matrix}_{length \times embedding\_size} \longrightarrow \text {Matrix}_{NEW\_length \times NEW\_embedding\_size} \longrightarrow \text {Vector}_{result\_size} $$


7. решение конечной задачи

\*) Векторное представление (обычно текста) еще называют **эмбеддинг**. Соответствующая таблица называется таблицей представлений или таблицей эмбеддингов.

\**) размер эмбеддинга либо является гиперпараметром, который мы подбираем во время обучения сети, либо он уже задан, если мы пользуемся готовыми предобученными эмбеддингами. Размер матрицы эмбеддингов равен $VocabSize \times EmbSize$, где VocabSize - размер словаря,  EmbSize - размер эмбеддинга. Когда мы составляем тензор для нашего текста, мы из матрицы эмбеддингов берём вектора для всех слов и получаем тензор размером  $LenText \times EmbSize$, где LenText - размер текста.

\***) тензор - это общее название для многомерных математических объектов из линейной алгебры. Примеры тензоров - скаляры или числа (0 измерений),  векторы (1 измерение), матрицы (2 измерения)


Кратко:

1. Подготовка текста (разбить на элементы, построить словать, перенумеровать)
2. Получение базовых векторых представлений (эмбеддинги слов и символов, построение матрицы текста)
3. Преобразование матрицы текста (учет локального контекста)
4. Агрегация (весь текст сжимается в вектор)
5. Решение конечной задачи

Далее в этой части акцент на пп. 2-4

# <a id='toc2_'></a>[Дистрибутивная семантика и векторные представления слов](#toc0_)

Дистрибути́вная сема́нтика — это область лингвистики, которая занимается вычислением степени семантической близости между лингвистическими единицами на основании их распределения (дистрибуции) в больших массивах лингвистических данных (текстовых корпусах).

NB: текст сначала разбивается на базовые элементы — символы или токены, они нумеруются, заменяются на идентификаторы, а затем по этим идентификаторам мы вытаскиваем из таблицы эмбеддингов нужные вектора. 

Рассмотрим, какие способы построения такой таблицы существуют. Раз уж мы работаем с нейросетями (дифференцируемыми моделями, которые настраиваются градиентным спуском), то самый естественный вариант — это считать таблицу эмбеддингов параметрами нейросети и обучать вместе с остальной сетью.

Так как этот способ использует обучение с учителем, у него есть существенный недостаток: нам нужна очень большая размеченная обучающая выборка, в которой будет достаточно разнообразная лексика, чтобы настроить эмбеддинги всех слов и чтобы модель в итоге хорошо работала на новых данных. 

**Проблема**: распределение Ципфа - нужно набрать достаточную выборку в т.ч. и для редких слов, поэтому объем нужно обучающей выборки для увеличения качества растет экспоненциально.

**Преимущество**: обучение с учителем — это наиболее стабильная и практичная постановка задачи машинного обучения. Надо стремиться всегда все задачи сводить к обучению с учителем. 

Для этого нам нужно найти разметку, то есть некоторую целевую переменную, которую мы будем предсказывать. Оказывается, в текстах такая целевая переменная существует: это **следующее слово**. Например, если вы увидите фрагмент фразы "шла Саша по...", какое следующее слово вы будете ожидать? Скорее всего, многие из вас будут ожидать слово "шоссе". Задача предсказания следующего слова по известному началу фразы называется **"моделированием языка"**.

**Упрощенный вариант задачи моделирования языка и есть задача дистрибутивной семантики**

В этом случае, входом нашей модели является не целая фраза, а одно лишь слово — обозначим его "x". Целевых переменных сразу несколько — это вероятности употребления всех остальных слов рядом со словом "x". Такие модели относятся к отрасли дистрибутивной семантики.

Тут не требуется нейросеть. Просто линейная регрессия по вектору, характеризующему соседние слова на некоторую глубину, предсказать целевое слово.


### <a id='toc2_1_1_'></a>[Дистрибутивная гипотеза](#toc0_)

You shall know the word by the company it keeps (Firrh, 1957)

**Смысл слова**, в значительной мере, описывается тем контекстом, в котором слово обычно употребляется

**Контекст** можно понимать по-разному. Чаще всего, это просто соседние слова — соседние в предложении, или соседние в целом документе.

**Ключевой объект**, характеризующий контекст — это матрица совместной встречаемости слов.

**Элементы** этой матрицы — это счётчики, то есть натуральные числа, обозначающие количество раз, когда два слова встретились в рамках одного контекста.


*Особенности:*

Размер словаря для достаточно большого корпуса текстов имеет порядок сотен тысяч. Матрица встречаемости — квадратная и симметричная, то есть в ней — порядка 10 в десятой степени элементов.

Матрица очень разреженная, то есть почти все её элементы нулевые и их можно хотя бы не хранить.

Есть небольшое количество слов (союзов, предлогов общеупотребимой лексики), которые встречаются намного чаще, чем все остальные слова. Значение матрицы в строках и столбцах соответствующих таким словам, существенно выше, чем в остальных. Это создаёт некоторый дисбаланс. 


Итак, матрица гигантская, распределение элементов сильно скошенное. Как же с этим всем работать?



### <a id='toc2_1_2_'></a>[Общий алгоритм](#toc0_)

1. Строится матрица совместной встречаемости слов

$$X_{|Vocabulary| \times |Vocabulary|}$$

- иногда можно строить целиком, но есть итерационные методы, позволяющие эффективно генерить на ходу по кускам
2. Сглаживание матрицы

- чтобы значения частотных слов не выделялись (видимо обратная функция к закону Ципфа)

3. Сжатие матрицы - построить достаточно хорошую аппроксимацию - обычно ищется разложение на произведение двух матриц меньшего ранга (факторизация)

$$X \sim W_{|Vocabulary| \times EmbSize} \cdot D_{EmbSize \times |Vocabulary|}$$

- ранг полученного $X$ будет не выше $EmbSize$ 
- часть информации потеряется
- можно дальше использовать только матрицу $W$

#### <a id='toc2_1_2_1_'></a>[Сглаживание](#toc0_)

Нужно, чтобы привести распределение элементов матрицы X к менее скошенному виду и уменьшить диапазон значений, уменьшить влияние частотных слов.

- счетчики могут отличаться на несколько порядков
- частотные слова искажают смысл
- логарифмическое сглаживание: $\log (1 - x_{ij}) \rightarrow$ переход от значений счетчиков к из порядкам
- точечная взаимная информация PPMI - определяется через энтропию и условную энтропию двух случайных величин

#### <a id='toc2_1_2_2_'></a>[Разложение](#toc0_)

Для поиска разложения $X$ используются следующие подходы:

1. Все элементы матрицы по определению положительные. Можно использовать методы из области **неотрицательного матричного разложения**.

- помимо LU-, LDU- и QR-разложений, есть еще несколько десятков более менее ходовых видов факторизации матриц.

2. Регрессия
- минимизация "ошибки восстановления"

$$\sum_{ij} \left ( Smooth(x_{ij} - w_{ik} \cdot d_{kj})\right )^2\rightarrow \min_{W,D}$$

3. Классификация
- подбор разложения из метода максимального правдоподобия, т.е. моделирование распределения вероятности встретить некоторое слово в контексте другого слова
- используется в `word2vec`, `fastText`

$$\sum_{CurWord} \sum_{CtxWord} P(CtxWord|CurWord; W,D) \rightarrow \max_{W,D}$$

4. Сингулярное разложение (SVD)

$$Smooth(X_{|V|\times|V|} = U_{|V|\times Z} \cdot \sum_{Z\times Z} \cdot V_{Z\times |V|}$$
$$W := U$$
$$D := \sum \cdot V$$
$\sum_{Z\times Z}$ - матрица, где по диагонали - сингулярные числа матрицы $Smooth(X)$  
U, V - ортогональные матрицы, т.е. $UU^T=VV^T=E$, что эквивалентно $U^{-1}=U^T$, $V^{-1}=V^T$, т.к. их столбцы - ортонормированные векторы. Еще одно свойство: $det(U) = det(V) = 1$

5. Вероятностные моделы со скрытыми переменными:
- латентное размещение Дирихле (LDA)

**Задача**

Вы построили матрицу совместной встречаемости с размером словаря 100000 (сто тысяч) и сжали её в две матрицы размерности $100000 \times 300$ и $300 \times 100000$. Сколько памяти Вы сэкономили этой операцией при том, что числа хранятся во float (4 байта)?

In [1]:
Gb = 1024 ** 3
size = 4

X = (10 ** 5) ** 2 
U = (10 ** 5) * 300
V = 300 * (10 ** 5)

print(f"До разложения:\t\t{X * size / Gb}\nПосле разложения:\t{(U+V) * size / Gb} ")
round((X - U - V) * size / Gb)

До разложения:		37.25290298461914
После разложения:	0.22351741790771484 


37

### <a id='toc2_1_3_'></a>[Модель GloVe (глобальные вектора)](#toc0_)

Была предложена в 2014 году.

**Акцент на глобальном контексте** - матрица совместной встречаемости строится для всего документа, а не для предложения или некотого другого подмножества текста.

Ищется разложение в рамках регрессии с квадратичным функционалом качества

$$\sum_{ij} \left ( Smooth(x_{ij} - w_{ik} \cdot d_{kj})\right )^2\rightarrow \min_{W,D}$$

- каждое слагаемое - ошибка восстановления по сжатой матрице частоты совместной встречаемости слов $i$ и $j$

Пока ниче интересного.

В качества сглаживания используется логарифм (все еще ниче интересного):

$$\sum_{ij} f(x_{ij}) \left ( \ln (1 + x_{ij}) - w_{ik} \cdot d_{kj})\right )^2\rightarrow \min_{W,D}$$

Добавляется весовая функция $f(x_{ij})$ (ну хоть чето...)

$$\begin{equation*}
f(x) = 
 \begin{cases}
   (x / x_{max})^{\alpha} & x \leq x_{max}\\
   1 & otherwise
 \end{cases}
\end{equation*}$$

- увеличивает веса редких сочетаний слов ("заводной апельсин", "логарифмический оргазм"), которые по идее должны отражать специфику текста
- $\alpha$ - чем меньше, тем отношение быстрее (более выпукло) стремится к 1, т.е. тем менее специфичным сочетаниям слов накидывается дополнительный вес в функционале

### <a id='toc2_1_4_'></a>[Модель word2vec](#toc0_)

Была предложена в 2013 году

В основе подхода — моделирование условного распределения вероятностей соседних слов.

**Акцент на локальном контексте** - скользящее окно из n токенов (n-граммы). На каждом шаге в центр окна ставится токен и оценивается по методу максимального правдоподобия вероятность его появления в этом окне вместе с остальными токенами окна, т.е. на каждом шаге мы обновляем параметры модели, чтобы повысить правдоподобие того, что мы сейчас наблюдаем.

Предложено два варианта модели:

- SkipGram - моделирует распределение соседей при условии центрального слова
$$\sum_{CenterW_{i}} P(CtxW_{-2}, CtxW_{-1}, CtxW_{+1}, CtxW_{+2}|CenterW_{i};W, D) \rightarrow \max_{W, D}$$
- CBOW - моделирует распределение центрального слова при условии известных соседей
$$\sum_{CenterW_{i}} P(CenterW_{i}|CtxW_{-2}, CtxW_{-1}, CtxW_{+1}, CtxW_{+2};W, D) \rightarrow \max_{W, D}$$

Т.е. для каждого слова хранятся и настраиваются два вектора (центральный и контекстный).

Параметры этой модели настраиваются градиентным спуском. По сути, процесс обучения word2vec идентичен обучению обычной нейросети, когда подаются обучающие примеры (в данном случае окна) один за другим, и после наблюдений небольшой пачки примеров веса модели обновляются.

Чего же использовать в качестве эмбеддингов слов (вектора же два, правда оба они нужны по сути только на этапе построения). Тут возможны варианты, популярными из которых являются следующие:
- брать только центральные вектора
- брать полусумму центрального и контектстного векторов

#### <a id='toc2_1_4_1_'></a>[Рассмотрим более подробно SkipGram](#toc0_)

Мы моделируем соседние слова в окне при условии известного центрального слова. 

В отличие от `GloVe`, суммирование идёт не по всем уникальным словам, а по всем возможным окнам в корпусе, то есть по всем словоупотреблениям — каждое словоупотребление на одном из шагов становится центром окна.

Берём контекст в некотором окне, оцениваем его правдоподобие и обновляем вес. Что это значит:

- исходим из того, что соседние слова условно независимы друг от друга (зависят только от центрального слова)

$$P(CtxW_{-2}, CtxW_{-1}, CtxW_{+1}, CtxW_{+2}|CenterW_{i};W, D) = \prod_j P(CtxW_{j}|CenterW_{i};W, D)$$

- то есть сложное распределение представляется в виде произведения простых распределений (факторизуется), которые моделируются по-отдельности (с помощью sotfmax)

$$P(CtxW_{j}|CenterW_{i};W, D) = \frac {e^{w_i \cdot d_j }}{\sum_{j=1}^{|V|} e^{w_i \cdot d_j }} = softmax$$

- внутри софтмакса мы используем скалярное произведение вектора текущего центрального слова с векторами остальных слов. Учитывая, что это нужно пересчитывать на каждом шаге градиентного спуска, получается много вычислений

- поэтому на практике часто аппроксиммируют софтмакс более [дешёвыми вариантами](https://ruder.io/word-embeddings-softmax/index.html) — один такой способ называется "отрицательным сэмплированием" (или "negative sampling"). Идея в том, что сумму в знаменателе мы считаем не по всему словарю, а по небольшому числу случайно выбранных слов. Эти слова мы выбираем каждый раз заново.

$$P(CtxW_{j}|CenterW_{i};W, D) \sim \frac {e^{w_i \cdot d_j^+ }}{\sum_{j=1}^{k} e^{w_i \cdot d_j^- }}, k \ll |V|$$

Проблема снижения вычислительной сложности софтмакса является очень насущной, поэтому были предложены и другие варианты — например, [иерархический софтмакс](https://towardsdatascience.com/hierarchical-softmax-and-negative-sampling-short-notes-worth-telling-2672010dbe08)


#### <a id='toc2_1_4_2_'></a>[Negative Sampling](#toc0_)
Коротко обсудим, как устроен алгоритм **Negative Sampling** в методе SkipGram:

Вычисление функции $\frac{e^{w_i \cdot d_j}}{\sum_{j=1}^{|V|}e^{w_i \cdot d_j}}$ по всему словарю сводится к задаче отличать случаи, когда следующее за последовательностью $c_i$ слово $w_i$ взято из обучающей выборки, а когда - выбрано наугад из словаря.

Вероятность события "слово $w_i$ следующее за контекстом $c_i$ взято из обучающей выборки" (контекст это пачка слов со своими индексами) обозначим как $P(y=1|w_i,c_i)$.

Вероятность того, что следующее слово выбрано наугад из словаря:  $P(y=0|\tilde{w}_{ij},c_i)$

Авторы метода предлагают оптимизировать следующую функцию:

$$J_θ = \sum_{w_i∈V}[logP(y=1|w_i,c_i) + \sum_{j=1}^{k}logP(y=0|\tilde{w}_{ij},c_i)] \to \max$$

В своей статье word2vec Томаш Миколов определяет указанные выше вероятности как сигмоиду от скалярного произведения центрального вектора слова $w_i$ и контекстного вектора слова $c_i$ 

$$P(y=1|w_i,c_i) = \sigma((w_i,d_j)) \\ P(y=0|\tilde{w_i},c_i) = 1-\sigma((\tilde{w_i},d_j)) = \sigma((-\tilde{w_i},d_j))$$

Таким образом вычисление softmax в SkipGram сводится к вычислению Negative Sampling Loss:

$$J_θ = \sum_{w_i∈V}[log\sigma(w_i,c_i) + \sum_{j=1}^{k}log(1-\sigma(\tilde{w}_{ij},c_i))] \to \max$$

### <a id='toc2_1_5_'></a>[Модель fastText](#toc0_)

Предложена автором модели word2vec

Важной особенностью естественных языков является то, что слова могут принимать различные формы, при этом не меняя смысла. Ну, а мы же хотим чтобы наши вектора описывали именно смысл. Другими, словами мы хотим получить инвариантность к словоизменению.

Для этого всегда можно нормализовать текст:
- стемминг (ненадежно)
- лемматизация (сложно)

**fastText = word2vec на уровне n-грамм симоволов**

Предлагается использовать вместо n-грамм токенов/слов (word2vec) использовать n-граммы символов.

- используются только достаточно частотные n-граммы, редкие игнорируются
- часть n-грамм может совпадать со словами/корнями (информация -> инфо, нфор, форм, ..., ация)
- выбирается центральное слово в некотором окне
- берем все n-граммы в этом слове и делаются градиентные шаги, аналогичные word2vec

Это позволяет получить **вектора слов, которых небыло в обучающей выборке**:

- мы просто берём все возможные N-граммы, которые встречаются в слове, для которого нам нужно получить вектор, берём полученные вектора N-грамм, которые насчитались, берём среднее. 

Еще некоторые интересные свойства такой модели:
- мы можем измерять сходство смыслов слов через скалярное произведение их векторов
    - к сожалению, это же означает, что вектор слова будет представлять единственный, наиболее частый смысл (например, "язык", скорее всего, будет только в смысле разговорного, но не как орган тела)
$$Similarity(i, j) = w_i \cdot w_j$$
- можно выбрать несколько слов и посмотреть графически, как они соотносятся друг с другом

- мы можем складывать и вычитать вектора слов, при этом, как бы, переходя по семантическим связям (это работает не для всех отношений не для всех слов, но для некоторых частотных — работает)
    - например, если мы вычтем из вектора для слова "женщина" вектор для слова "мужчина", а потом прибавим вектор для слова "дядя" и попробуем поискать ближайшие вектора к полученному, то скорее всего среди них найдем слово "тётя"



#### <a id='toc2_1_5_1_'></a>[Общий алгоритм обучения FastText Skip Gram Negative Sampling выглядит следующим образом:](#toc0_)

1. Очистить и токенизировать обучающую коллекцию документов
2. Построить словарь - подсчитать частоты всех целых токенов и N-грамм заданной длины (например, от 3 до 6 символов). При построении словаря раз в заданное число шагов прореживать словарь - удалить из словаря токены, набравшие с предыдущего прореживания меньше всего употреблений (или меньше заданного порога).
3. Проход по корпусу скользящим окном заданной ширины, для каждой позиции окна выполнять шаги 4-7.
4. Для текущего словоупотребления в центре окна выделить его N-граммы, содержащиеся в словаре (то есть только достаточно частотные N-граммы)
5. Вычислить вектор центрального токена, усреднив вектора целого токена (если он есть в словаре) и всех N-грамм, выделенных на шаге 4.
6. Выбрать случайным образом отрицательные слова (сделать negative sampling).
7. Обновить следущие вектора так, чтобы улучшить оценку правдоподобия:
- N-грамм, участвовавших в получении вектора центрального токена,
- контекстные вектора всех токенов в окне, кроме центрального,
- контекстные вектора отрицательных слов.
8. Повторять шаги 3-7 заданное число раз или до сходимости.

NB! FastText учитывает само центральное слово как n-грамму, только если оно достаточно частотное


В этой лекции мы поговорили о векторизации текстов. Она может выполняться на уровне символов, целых токенов или N-грамм символов. Больше внимания мы уделили способам получения векторов слов, поговорили о некоторых методах дистрибутивной семантики, рассмотрели три популярные модели: GloVe, word2vec и FastText, а также поговорили о том, что можно делать с векторами слов даже без нейросетей — например, искать ближайшие слова или делать пусть грубую, но какую-то арифметику смыслов.

Семинары в директории `./stepic-dl-nlp`

# <a id='toc3_'></a>[Теоретические вопросы: Дистрибутивная семантика](#toc0_)

Вы обучаете Word2Vec Skip Gram Negative Sampling с окном заданной ширины. Например, окно размерности 5 подразумевает, что положительными примерами считаются слова, отстоящие от центрального слова не более чем на 2 позиции влево или вправо. Центральное слово не учитывается как контекстное слово.

Напишите функцию, которая **генерирует обучающие примеры из текста**. Каждый обучающий пример должен иметь вид кортежа из трёх элементов (CenterWord, CtxWord, Label), где $CenterWord \in \mathbb{N}$ - идентификатор токена в центре окна, $CtxWord \in \mathbb{N}$ - идентификатор соседнего токена, $Label \in \{0, 1\}$ если CtxWord настоящий и 0, если это отрицательный пример.

Функция должна возвращать список обучающих примеров.

Аргумент ns_rate задаёт количество отрицательных примеров, которое нужно сгенерировать на каждый положительный пример. При семплировании отрицательных слов обычно не проверяют, встречается ли это слово в окне. Таким образом, среди отрицательных примеров могут появиться положительные.

Входной текст уже токенизирован, токены заменены на их идентификаторы.

Тесты генерируются случайно, ограничения:

- len(text) < 20
- window_size <= 11, нечётное
- vocab_size < 100
- ns_rate < 3

Слова имеют идентификаторы 0..vocab_size - 1 (как возвращает np.random.randint).

Обратите также внимание на то, что -3 // 2 != -(3 // 2).

In [2]:
import sys
import ast
import numpy as np

SAMPLES = "[1, 0, 1, 0, 0, 5, 0, 3, 5, 5, 3, 0, 5, 0, 5, 2, 0, 1, 3]\n3\n6\n1", 
READER = (x for x in SAMPLES[0].split('\n')); 
sys.stdin.readline = lambda: next(READER)       # тупо шоп не переписывать

def parse_array(s):
    return np.array(ast.literal_eval(s))

def read_array():
    return parse_array(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))

def make_diag_mask(size, radius):
    mask = np.zeros(size)
    for i in range(-radius, radius + 1):
        mask = mask + np.eye(size, k=i)
    return mask - np.eye(size)

def generate_w2v_sgns_samples(text, window_size, vocab_size, ns_rate):
    """
    text - list of integer numbers - ids of tokens in text
    window_size - odd integer - width of window
    vocab_size - positive integer - number of tokens in vocabulary
    ns_rate - positive integer - number of negative tokens to sample per one positive sample

    returns list of training samples (CenterWord, CtxWord, Label)
    """
    masks =  make_diag_mask(len(text), (window_size - 1) // 2)

    res = []
    for center_word, mask in zip(text, masks):
        ctx_positive = text[mask.nonzero()]
        for ctx_word in ctx_positive:
            res.append(np.array([center_word, ctx_word,  1]))
            for i in range(ns_rate):
                # не понятно, надо ли исключать из негативного контекста центральное слово
                ctx_neg = np.random.randint(vocab_size - 1)
                # в генсиме вроде исключается, но проверялка пропускает и так, и так
                ctx_neg = ctx_neg if ctx_neg < center_word else ctx_neg + 1
                res.append(np.array([center_word, ctx_neg,  0]))
    return res


text = read_array()
window_size = int(sys.stdin.readline().strip())
vocab_size = int(sys.stdin.readline().strip())
ns_rate = int(sys.stdin.readline().strip())

result = generate_w2v_sgns_samples(text, window_size, vocab_size, ns_rate)

write_array(np.array(result))

[[1, 0, 1], [1, 2, 0], [0, 1, 1], [0, 2, 0], [0, 1, 1], [0, 3, 0], [1, 0, 1], [1, 4, 0], [1, 0, 1], [1, 3, 0], [0, 1, 1], [0, 2, 0], [0, 0, 1], [0, 2, 0], [0, 0, 1], [0, 2, 0], [0, 5, 1], [0, 4, 0], [5, 0, 1], [5, 3, 0], [5, 0, 1], [5, 4, 0], [0, 5, 1], [0, 1, 0], [0, 3, 1], [0, 5, 0], [3, 0, 1], [3, 4, 0], [3, 5, 1], [3, 5, 0], [5, 3, 1], [5, 1, 0], [5, 5, 1], [5, 4, 0], [5, 5, 1], [5, 1, 0], [5, 3, 1], [5, 0, 0], [3, 5, 1], [3, 0, 0], [3, 0, 1], [3, 2, 0], [0, 3, 1], [0, 2, 0], [0, 5, 1], [0, 3, 0], [5, 0, 1], [5, 4, 0], [5, 0, 1], [5, 3, 0], [0, 5, 1], [0, 5, 0], [0, 5, 1], [0, 4, 0], [5, 0, 1], [5, 3, 0], [5, 2, 1], [5, 3, 0], [2, 5, 1], [2, 5, 0], [2, 0, 1], [2, 3, 0], [0, 2, 1], [0, 3, 0], [0, 1, 1], [0, 1, 0], [1, 0, 1], [1, 5, 0], [1, 3, 1], [1, 0, 0], [3, 1, 1], [3, 1, 0]]


Вы обучаете Word2Vec Skip Gram Negative Sampling.

Напишите функцию, обновляющую веса модели при получении одного обучающего примера в формате (CenterWord, CtxWord, Label).

При обучении предсказания модели вычисляются по формуле $P(CtxWord | CenterWord) = \sigma(W_{CenterWord, :} \cdot D_{CtxWord,:})$.

Функция потерь - бинарная кросс-энтропия (BCE).

**Алгоритм:**

У нас на вход подаются два вектора w и d. Нам необходимо обновить их значения в ячейках center_word и context_word. Для этого посчитает градиент от ВСЕ, который равняется: z*(sigmoid-y), где z - это параметр w или d соответственно, а y = label. Чтобы рассчитать новые параметры для x и w необходимо от старых значений отнять градиенту, умноженные на learning_rate.

Фунция потерь:
$$BCE = −L \log \hat y −(1−L) \log (1− \hat y)$$

где L − метка класса, $\hat y$ − prediction

Возьмем вектор весов центрального слова, номер которого введен в условии. В sample_input введено 2, и возьмем из этого вектора первый вес $w_{21}$. Будем обновлять его градиентным спуском. Например:

$$w_{21} = w_{21_0} - BCE'_{w_{21}} \cdot learning\_rate$$

И так происходит с каждым весом в эмбеддинге центрального слова, номер которого подан на вход (в данном случае 2). При этом, например:

$$BCE'_{w_{21}} = (\sigma - L) \cdot d_{15}$$

(контекстное слово пришло под номером 5 в тестовом примере)

**Веста второго вектора обновлять на основе первого вектора ДО ОБНОВЛЕНИЯ!!!!!!!!**

In [3]:
SAMPLES = """[[0.3449417709491044, 0.6762047256081501, 0.9583446027893963], [0.6247126159157468, 0.22038323197740317, 0.29717611444948355], [0.9836099232994968, 0.3847689688960674, 0.033312247867206435], [0.4217704869846559, 0.0023859008971685025, 0.009686915033163657], [0.6933070658521228, 0.9705089533296152, 0.9189360293193337], [0.024858486425111903, 0.11331113152689753, 0.6492144300167894], [0.7861289466352543, 0.227319130535791, 0.8165251907260063], [0.7672181161105678, 0.04865001026002924, 0.07514404284170773]]\n[[0.4628817426583818, 0.7747296319956671, 0.1374808935513827], [0.17026823169513283, 0.4094733988461122, 0.3175531656197459], [0.2910876746161247, 0.6340566555548147, 0.23158010794029804], [0.8449042648180852, 0.4796593509107806, 0.11278090182290745], [0.049097778744511156, 0.6254116250148337, 0.13038703647472905], [0.882545488649187, 0.6223076699449618, 0.1633041302523962], [0.6704032810194875, 0.941803340812521, 0.7358646489592193], [0.9875878745059805, 0.17935677165390562, 0.6798846454394736]]\n2\n5\n0\n0.342405260598321""", 
READER = (x for x in SAMPLES[0].split('\n')); 
sys.stdin.readline = lambda: next(READER)       # тупо шоп не переписывать

def update_w2v_weights(center_embeddings, context_embeddings, center_word, context_word, label, learning_rate):
    """ Update center_embeddings and context_embeddings inplace:
    center_embeddings - VocabSize x EmbSize
    context_embeddings - VocabSize x EmbSize
    center_word - int - identifier of center word
    context_word - int - identifier of context word
    label - 1 if context_word is real, 0 if it is negative
    learning_rate - float > 0 - size of gradient step
    """
    center_w = center_embeddings[center_word]
    ctx_w = context_embeddings[context_word]
    sigmoida = 1 / (1 + np.exp(-sum(center_w * ctx_w)))

    new_center_w = center_w - ctx_w * (sigmoida - label) * learning_rate
    new_ctx_w = ctx_w - center_w * (sigmoida - label) * learning_rate

    center_embeddings[center_word] = new_center_w
    context_embeddings[context_word] = new_ctx_w

center_embeddings = read_array()
context_embeddings = read_array()
center_word = int(sys.stdin.readline().strip())
context_word = int(sys.stdin.readline().strip())
label = int(sys.stdin.readline().strip())
learning_rate = float(sys.stdin.readline().strip())

update_w2v_weights(center_embeddings, context_embeddings,
                   center_word, context_word, label, learning_rate)

write_array(center_embeddings)
write_array(context_embeddings)

[[0.3449417709491044, 0.6762047256081501, 0.9583446027893963], [0.6247126159157468, 0.22038323197740317, 0.29717611444948355], [0.7561584406822225, 0.22438652516534294, -0.008774836618697823], [0.4217704869846559, 0.0023859008971685025, 0.009686915033163657], [0.6933070658521228, 0.9705089533296152, 0.9189360293193337], [0.024858486425111903, 0.11331113152689753, 0.6492144300167894], [0.7861289466352543, 0.227319130535791, 0.8165251907260063], [0.7672181161105678, 0.04865001026002924, 0.07514404284170773]]
[[0.4628817426583818, 0.7747296319956671, 0.1374808935513827], [0.17026823169513283, 0.4094733988461122, 0.3175531656197459], [0.2910876746161247, 0.6340566555548147, 0.23158010794029804], [0.8449042648180852, 0.4796593509107806, 0.11278090182290745], [0.049097778744511156, 0.6254116250148337, 0.13038703647472905], [0.6290474670186392, 0.5231442006778062, 0.15471882755224034], [0.6704032810194875, 0.941803340812521, 0.7358646489592193], [0.9875878745059805, 0.17935677165390562, 0.679

In [4]:
center = np.array([0.9836099232994968, 0.3847689688960674, 0.033312247867206435])
ctx = np.array([0.882545488649187, 0.6223076699449618, 0.1633041302523962])

sig = 1 / (1 + np.exp(-sum(center * ctx)))
learning_rate = 0.342405260598321
label = 0

tmp = center
center = center - ctx * (sig - label) * learning_rate
ctx = ctx - tmp * (sig - label) * learning_rate

center, ctx    
# [0.7561584406822225, 0.22438652516534294, -0.008774836618697823] 
# [0.6290474670186392, 0.5231442006778062, 0.15471882755224034]

(array([ 0.75615844,  0.22438653, -0.00877484]),
 array([0.62904747, 0.5231442 , 0.15471883]))

Вы обучаете FastText SkipGram Negative Sampling с окном заданной ширины (аналогично первой задаче). Общий алгоритм обучения также описывался ранее. Единственное существенное отличие от Word2Vec - использование N-грамм для получения вектора центрального слова.

Напишите функцию, которая генерирует обучающие примеры из текста. Каждый обучающий пример должен иметь вид кортежа из трёх элементов (CenterSubwords, CtxWord, Label), где CenterSubwords - список идентификаторов N-грамм, входящих в токен в центре окна (включая идентификатор самого токена), $CtxWord \in \mathbb{N}$ - идентификатор соседнего токена, $Label \in \{0, 1\}$ - 1 если CtxWord настоящий и 0, если это отрицательный пример.

Функция должна возвращать список обучающих примеров.

Аргумент ns_rate задаёт количество отрицательных примеров, которое нужно сгенерировать на каждый положительный пример.

Входной текст уже токенизирован, токены заменены на их идентификаторы.

Для получения списка N-грамм, входящих в токен, используется заранее построенный словарь (он передаётся в функцию, которую Вам нужно написать). В этом задании следует считать, что сам токен всегда есть в словаре и поэтому нужно обновлять вектор для него (хотя при обучении FastText на настоящих данных это может быть не так). Все n-граммы имеют номера больше или равные vocab_size.

Тесты генерируются случайно, ограничения:

    len(text) < 20
    vocab_size < 10
    window_size <= 11, нечётное
    ns_rate < 3
    ngrams_n < 20

Слова имеют идентификаторы 0..vocab_size - 1 (как возвращает np.random.randint). N-граммы имеют идентификаторы vocab_size .. vocab_size + ngrams_n - 1.

Обратите также внимание на то, что -3 // 2 != -(3 // 2).

In [5]:
SAMPLES = """[1, 2, 0, 1, 4, 0, 4, 1, 5, 4, 5, 4, 5, 1]\n3\n6\n2\n[[17], [10, 12], [20, 20], [7, 13], [], [7, 11]]""", 
READER = (x for x in SAMPLES[0].split('\n')); 
sys.stdin.readline = lambda: next(READER)       # тупо шоп не переписывать


def read_list():
    return ast.literal_eval(sys.stdin.readline())

def make_diag_mask(size, radius):
    mask = np.zeros(size)
    for i in range(-radius, radius + 1):
        mask = mask + np.eye(size, k=i)
    return mask - np.eye(size)

def generate_ft_sgns_samples(text, window_size, vocab_size, ns_rate, token2subwords):
    """
    text - list of integer numbers - ids of tokens in text
    window_size - odd integer - width of window
    vocab_size - positive integer - number of tokens in vocabulary
    ns_rate - positive integer - number of negative tokens to sample per one positive sample
    token2subwords - list of lists of int - i-th sublist contains list of identifiers of n-grams for token #i (list of subword units)

    returns list of training samples (CenterSubwords, CtxWord, Label)
    """
    masks =  make_diag_mask(len(text), (window_size - 1) // 2)

    res = []
    for center_word, mask in zip(text, masks):
        ctx_positive = text[mask.nonzero()]
        for ctx_word in ctx_positive:
            сenter_subwords = np.hstack((center_word, token2subwords[center_word])).tolist()

            res.append([сenter_subwords, ctx_word,  1])
            for i in range(ns_rate):
                # не понятно, надо ли исключать из негативного контекста центральное слово
                ctx_neg = np.random.randint(vocab_size - 1) # исключим...
                # в генсиме вроде исключается, но проверялка пропускает и так, и так
                ctx_neg = ctx_neg if ctx_neg < center_word else ctx_neg + 1
                res.append([сenter_subwords, ctx_neg,  0])

    return res


text = read_array()
window_size = int(sys.stdin.readline().strip())
vocab_size = int(sys.stdin.readline().strip())
ns_rate = int(sys.stdin.readline().strip())
token2subwords = read_list()

result = generate_ft_sgns_samples(text, window_size, vocab_size, ns_rate, token2subwords)

print(repr(result))

[[[1, 10, 12], 2, 1], [[1, 10, 12], 2, 0], [[1, 10, 12], 5, 0], [[2, 20, 20], 1, 1], [[2, 20, 20], 1, 0], [[2, 20, 20], 0, 0], [[2, 20, 20], 0, 1], [[2, 20, 20], 3, 0], [[2, 20, 20], 5, 0], [[0, 17], 2, 1], [[0, 17], 2, 0], [[0, 17], 4, 0], [[0, 17], 1, 1], [[0, 17], 2, 0], [[0, 17], 3, 0], [[1, 10, 12], 0, 1], [[1, 10, 12], 0, 0], [[1, 10, 12], 0, 0], [[1, 10, 12], 4, 1], [[1, 10, 12], 4, 0], [[1, 10, 12], 0, 0], [[4.0], 1, 1], [[4.0], 2, 0], [[4.0], 5, 0], [[4.0], 0, 1], [[4.0], 0, 0], [[4.0], 1, 0], [[0, 17], 4, 1], [[0, 17], 4, 0], [[0, 17], 5, 0], [[0, 17], 4, 1], [[0, 17], 4, 0], [[0, 17], 1, 0], [[4.0], 0, 1], [[4.0], 2, 0], [[4.0], 0, 0], [[4.0], 1, 1], [[4.0], 0, 0], [[4.0], 0, 0], [[1, 10, 12], 4, 1], [[1, 10, 12], 5, 0], [[1, 10, 12], 3, 0], [[1, 10, 12], 5, 1], [[1, 10, 12], 0, 0], [[1, 10, 12], 0, 0], [[5, 7, 11], 1, 1], [[5, 7, 11], 0, 0], [[5, 7, 11], 4, 0], [[5, 7, 11], 4, 1], [[5, 7, 11], 0, 0], [[5, 7, 11], 4, 0], [[4.0], 5, 1], [[4.0], 2, 0], [[4.0], 0, 0], [[4.0], 5

Вы обучаете FastText SkipGram Negative Sampling.

Напишите функцию, обновляющую веса модели при получении одного обучающего примера в формате $(CenterSubwords, CtxWord, Label)$.

При обучении предсказания модели вычисляются по формуле 
$$P(CtxWord | CenterSubwords) = \sigma\left(\left(\sum_{w \in CenterSubwords}^{len(CenterSubwords)} \frac {W_{w, :}} {len(CenterSubwords)} \right) \cdot D_{CtxWord,:}\right)$$ 

Функция потерь - бинарная кросс-энтропия.

Похоже на то что делалось выше для w2c, только:

- Усредняем векторы эмбедингов center_subwords.
- Считаем дельту уменьшения center_embeddings[center_subwords] и делим её на len(center_subwords).
- В формуле обновления весов context_embeddings[context_word] используем усреднённый вектор эмбедингов center_subwords

Аналогично - все расчеты до обновления!

In [6]:
SAMPLES = """[[0.07217140995735816, 0.9807495045952024, 0.5888650678318127, 0.9419020475323008, 0.9698687137771355], [0.17481764801167854, 0.9598681333667267, 0.8615416075076997, 0.6649845254089604, 0.14272822189820067], [0.695257160390079, 0.6252124583357915, 0.788572884360212, 0.5407620598434707, 0.4742760619803522], [0.3720755825170682, 0.8734430653555122, 0.29388553936147677, 0.7833976055802006, 0.11647446813597206], [0.4793503066165381, 0.7731679392102295, 0.6466062364447424, 0.5834632727525674, 0.16975097768580916], [0.46855676928071344, 0.7440440871653314, 0.5968916205486556, 0.6949993371605877, 0.9995564750677164], [0.3995517204225809, 0.30217048674177027, 0.6934836340605662, 0.5025452046745376, 0.43990420866402447], [0.6233285824044058, 0.7510765715859197, 0.8764982899024905, 0.42892241183749247, 0.9241569354174014], [0.21063022873083803, 0.979366603599722, 0.07879437255385402, 0.7103116511451802, 0.298121842692622], [0.7991181799927396, 0.8700912396205017, 0.4936455488806514, 0.9306352063022928, 0.671689987782089], [0.11245515636577097, 0.2591385008756272, 0.38393130144123977, 0.5927928993875077, 0.3343301767582757], [0.027340724019638274, 0.15461071231349877, 0.7955192467457007, 0.050624838697975516, 0.26136570172628426], [0.7825083895933859, 0.9046538942978853, 0.4559636175207443, 0.733829258685726, 0.022174763292638677], [0.6968176063951074, 0.47974647096747125, 0.8885207189970179, 0.016167994434510558, 0.13260182909882334], [0.5947903955259933, 0.07459974351651177, 0.11391699485528617, 0.823474357110585, 0.4918622459339238], [0.6272760016913231, 0.2711994820963495, 0.24338892914238242, 0.7731707300677505, 0.03720128542002399], [0.8640858092433228, 0.027663971153230382, 0.9271422334467209, 0.37457369227183035, 0.17413436429736662], [0.4878584763813121, 0.5022845803948351, 0.13899660663745628, 0.8353408935052742, 0.48314336609381436], [0.8197910105251979, 0.5371430936015362, 0.12965724315376936, 0.06244349080403733, 0.9558816248633216], [0.5929477505385994, 0.36687167726065173, 0.42925321480480627, 0.8435274356179648, 0.8550018469714032], [0.45785273815309, 0.008764229829187009, 0.6840407156586629, 0.04831125277736026, 0.14609911971743395], [0.1579479219010974, 0.1298470924838635, 0.8283362978065627, 0.9140741421274726, 0.7516395431217443], [0.01139316661353773, 0.6980229640742956, 0.45528806869472405, 0.7653990849713008, 0.24848012670857944], [0.8750941097872984, 0.6964598870452183, 0.6675389863133752, 0.391939718013135, 0.30592620271209714], [0.024161748164975072, 0.6512328549928654, 0.27784751504029503, 0.32588414662648524, 0.4073676483413957], [0.7372935688667617, 0.9743689028772393, 0.26179932035274445, 0.3556999822154028, 0.8234406534181563], [0.9358431512408416, 0.0030942521035778325, 0.7052198210371732, 0.3494249594704901, 0.06494462197366668], [0.027642224051125597, 0.45820907093457997, 0.6172763215932299, 0.03520578036716404, 0.05004091043245007]]
[[0.3619192935809462, 0.7910582560833153, 0.173840770588212, 0.8486217599360419, 0.09895998679198104], [0.9524670374363299, 0.577316446205222, 0.3348594666828074, 0.7987547183235284, 0.710457681490417], [0.8400820704952479, 0.9414962586451427, 0.08399082278691339, 0.425927381574433, 0.6304514720560764], [0.5331686510681622, 0.2751366715811131, 0.8329999135745643, 0.2770290564458684, 0.020564166091874392], [0.9852792048968001, 0.922320208232837, 0.7297936992308128, 0.20212997935663524, 0.5277458149323955], [0.43383566311415755, 0.14151987203148808, 0.3267585826852797, 0.8796734627573763, 0.14253685112772174], [0.24559727482999572, 0.3015598034026842, 0.12351719983998721, 0.6141130319406622, 0.9210871618079258], [0.21915908704207665, 0.9809645232509783, 0.8685879466971278, 0.9956335594634693, 0.0441562419906687], [0.24988758739587902, 0.42298807118368675, 0.01922872769211703, 0.02806386746602596, 0.2821901214584819], [0.43997555452635384, 0.5078839449569567, 0.812607950040521, 0.9998014106280365, 0.1559607489614684], [0.9092151190046189, 0.5930002929595868, 0.315159378929991, 0.4052299042409616, 0.984475831988958], [0.7836990450026143, 0.002466529016497798, 0.8465916260137056, 0.7227126698344118, 0.5087557482398855], [0.4125921074144525, 0.5582795115000383, 0.889307828978137, 0.928416977596577, 0.8437462138575066], [0.11810981794872477, 0.07787452990697508, 0.3907338451314212, 0.6841828899516664, 0.4547615738832046], [0.4977766315279062, 0.09878866849137813, 0.0622140049250518, 0.9008881823827194, 0.3694055807903669], [0.12415427540834822, 0.01064247175537103, 0.1439469061372417, 0.43996173718103593, 0.3846553735294024], [0.36544315427420426, 0.6651402425072226, 0.3837201693785094, 0.54713466624535, 0.6925194086063208], [0.8217730539154436, 0.7380601103419114, 0.4790971996703556, 0.935248458815274, 0.6385239169547122], [0.4884363834477089, 0.783319748626155, 0.018212966919229467, 0.03662832627793777, 0.03532160993715294], [0.6820505211290306, 0.25769913167047753, 0.9677388106523852, 0.4471332422618759, 0.7731319006564568], [0.3695513424667971, 0.5118113495291988, 0.1721439269100805, 0.09451631327113852, 0.8369170475041434], [0.7918542552021289, 0.0245240901264403, 0.6658133706965796, 0.9740885323982209, 0.02660284500887522], [0.5604137104962275, 0.5643917632639455, 0.6756476068355826, 0.9466913679034125, 0.21062462975598062], [0.7306868573812846, 0.7573083135261555, 0.9450278665003865, 0.9649869335038909, 0.1262321882978371], [0.6830284536315845, 0.7383035166437748, 0.7985226892860073, 0.005247820534787007, 0.6886083391552933], [0.6905561126225058, 0.3220803445510755, 0.8885006766287556, 0.32709316933290455, 0.9126547743770385], [0.26866358146648694, 0.9355232286537734, 0.5254946965960933, 0.6487428023364232, 0.9405298594379049], [0.33881123962516546, 0.6820622877451537, 0.3053828831926755, 0.9229486901650673, 0.5450270097149575]]\n[3]\n1\n1\n0.8562235244377375""", 
READER = (x for x in SAMPLES[0].split('\n')); 
sys.stdin.readline = lambda: next(READER)       # тупо шоп не переписывать


def read_list():
    return ast.literal_eval(sys.stdin.readline())

def parse_array(s):
    return np.array(ast.literal_eval(s))

def read_array():
    return parse_array(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))

def update_ft_weights(center_embeddings, context_embeddings, center_subwords, context_word, label, learning_rate):
    """
    center_embeddings - VocabSize x EmbSize
    context_embeddings - VocabSize x EmbSize
    center_subwords - list of ints - list of identifiers of n-grams contained in center word
    context_word - int - identifier of context word
    label - 1 if context_word is real, 0 if it is negative
    learning_rate - float > 0 - size of gradient step
    """
    center_ws = center_embeddings[center_subwords]  # пачка векторов
    mean_center_w = center_ws.mean(axis=0)          # один вектор
    ctx_w = context_embeddings[context_word]        # один вектор
    sigmoida = 1 / (1 + np.exp(-sum(mean_center_w * ctx_w)))    # скаляр
    weight = (sigmoida - label) * learning_rate                 # скаляр

    new_center_ws = center_ws - ctx_w * weight / len(center_subwords)
    new_ctx_w = ctx_w - mean_center_w * weight

    center_embeddings[center_subwords] = new_center_ws
    context_embeddings[context_word] = new_ctx_w


center_embeddings = read_array()
context_embeddings = read_array()
center_subwords = read_array()
context_word = int(sys.stdin.readline().strip())
label = int(sys.stdin.readline().strip())
learning_rate = float(sys.stdin.readline().strip())

update_ft_weights(center_embeddings, context_embeddings,
                  center_subwords, context_word, label, learning_rate)

write_array(center_embeddings)
write_array(context_embeddings)

[[0.07217140995735816, 0.9807495045952024, 0.5888650678318127, 0.9419020475323008, 0.9698687137771355], [0.17481764801167854, 0.9598681333667267, 0.8615416075076997, 0.6649845254089604, 0.14272822189820067], [0.695257160390079, 0.6252124583357915, 0.788572884360212, 0.5407620598434707, 0.4742760619803522], [0.5017594511598129, 0.9520480219915879, 0.3394785828834429, 0.8921526573535964, 0.21320737019064417], [0.4793503066165381, 0.7731679392102295, 0.6466062364447424, 0.5834632727525674, 0.16975097768580916], [0.46855676928071344, 0.7440440871653314, 0.5968916205486556, 0.6949993371605877, 0.9995564750677164], [0.3995517204225809, 0.30217048674177027, 0.6934836340605662, 0.5025452046745376, 0.43990420866402447], [0.6233285824044058, 0.7510765715859197, 0.8764982899024905, 0.42892241183749247, 0.9241569354174014], [0.21063022873083803, 0.979366603599722, 0.07879437255385402, 0.7103116511451802, 0.298121842692622], [0.7991181799927396, 0.8700912396205017, 0.4936455488806514, 0.93063520630

Вы собираетесь обучать GloVe и строите матрицу совместной встречаемости токенов, то есть матрицу, в ij ячейке которой стоит количество документов, в которых употреблялось и слово i и слово j. На главной диагонали должны стоять нули (то есть не нужно считать количество совместных употреблений слова с самим собой).

Напишите функцию, которая получает на вход список документов и строит такую матрицу. Документы уже токенизированы и токены представлены их числовыми идентификаторами.

Для хранения матрицы используйте scipy.sparse.dok_matrix.


In [20]:
import sys
import ast
import numpy as np
import scipy.sparse
from itertools import combinations

SAMPLES = """[[0, 2, 2, 2, 0, 0], [1, 1, 2, 1, 1], [2, 2, 1, 1]]\n3""", 
READER = (x for x in SAMPLES[0].split('\n')); 
sys.stdin.readline = lambda: next(READER)       # тупо шоп не переписывать


def read_array():
    return ast.literal_eval(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))


def generate_coocurrence_matrix(texts, vocab_size):
    """
    texts - list of lists of ints - i-th sublist contains identifiers of tokens in i-th document
    vocab_size - int - size of vocabulary
    returns scipy.sparse.dok_matrix
    """
    res = scipy.sparse.dok_matrix((vocab_size, vocab_size))
    for doc in text:
        doc_voc = set(doc)  # np.unique(np.array(doc))
        for comb in combinations(doc_voc, 2):
            res[comb] += 1

    return res + res.T


text = read_array()
vocab_size = int(sys.stdin.readline().strip())

result = generate_coocurrence_matrix(text, vocab_size)

write_array(result.toarray())

[[0.0, 0.0, 1.0], [0.0, 0.0, 2.0], [1.0, 2.0, 0.0]]


Вы обучаете GloVe. Матрица совместной встречаемости $X \in \mathbb{R}^{|Vocab| \times |Vocab|}$ построена.

Напишите функцию, которая обновляет параметры модели градиентным спуском (делает один градиентный шаг). Обратите внимание, что Вы должны изменить значения массивов w и d (inplace), а не создать новые массивы с новыми значениями.

Функция потерь: $GloVeLoss(W, D, X) = \sum_{ij} f(x_{ij})(\ln(1+x_{ij}) - W_{i,:} \cdot  D_{:,j})^2 \rightarrow \min_{W,D}$

\*) Слагаемое $W_{i,:} \cdot  D_{:,j}$, в формуле ф-ции потерь, это скалярное произведение i слова из набора эмбедингов W и j слова из набора DD

весовая функция $f(x_{ij})$

$$\begin{equation*}
f(x) = 
 \begin{cases}
   (x / x_{max})^{\alpha} & x \leq x_{max}\\
   1 & otherwise
 \end{cases}
\end{equation*}$$
Возможно, будет полезно пересмотреть фрагмент лекции про GloVe.

Тесты генерируются случайно.

Если $L =  f(X) ||W \cdot D^T - Y||^2$ ,то 

$$dL/dW = 2 f(X) (W \cdot D^T - Y) \cdot D$$

$$dL/dD = (2 f(X) (W \cdot D^T - Y))^T \cdot W$$

В нашем случае есть еще коэффициент f(x), который необходимо поэлементно умножить на содержимое скобок, а Y = ln(1 + X)

In [52]:
SAMPLES = """[[72, 67, 24, 81, 52, 43, 49, 12, 84, 77, 22, 66, 66, 0, 59, 4, 71, 78, 37, 69, 39, 63, 68, 36, 97, 17], [72, 38, 34, 43, 11, 46, 91, 96, 43, 4, 80, 77, 19, 18, 39, 8, 43, 58, 59, 43, 40, 55, 14, 96, 90, 43], [83, 82, 22, 93, 5, 17, 68, 30, 2, 67, 7, 8, 34, 2, 88, 66, 31, 52, 96, 13, 9, 83, 3, 9, 91, 15], [73, 25, 40, 82, 42, 30, 79, 77, 15, 76, 65, 6, 7, 44, 98, 88, 65, 74, 33, 48, 61, 54, 64, 28, 49, 38], [47, 41, 2, 54, 5, 13, 36, 0, 97, 15, 80, 90, 38, 27, 24, 31, 32, 20, 77, 20, 8, 11, 24, 19, 77, 23], [55, 26, 42, 5, 98, 87, 36, 1, 11, 19, 57, 68, 92, 49, 98, 9, 98, 24, 0, 13, 14, 90, 10, 51, 30, 30], [98, 12, 2, 66, 27, 12, 12, 60, 46, 71, 89, 82, 75, 49, 5, 77, 52, 96, 29, 32, 51, 71, 45, 16, 74, 12], [18, 92, 95, 62, 51, 65, 1, 49, 51, 62, 16, 64, 97, 48, 78, 14, 90, 50, 43, 49, 59, 11, 75, 50, 60, 2], [47, 45, 88, 78, 93, 30, 79, 20, 69, 68, 6, 76, 41, 3, 57, 98, 62, 6, 65, 53, 7, 9, 76, 96, 19, 88], [45, 73, 39, 70, 21, 62, 82, 13, 14, 72, 8, 23, 99, 49, 33, 80, 21, 67, 37, 31, 38, 48, 40, 61, 61, 67], [61, 86, 91, 61, 13, 88, 79, 56, 78, 87, 91, 94, 37, 14, 15, 44, 91, 3, 6, 23, 15, 85, 18, 58, 11, 4], [50, 28, 55, 44, 21, 62, 98, 64, 85, 84, 4, 31, 59, 16, 51, 11, 37, 44, 6, 60, 47, 54, 70, 29, 32, 74], [39, 6, 17, 54, 15, 71, 24, 94, 5, 16, 15, 74, 43, 98, 75, 10, 79, 78, 99, 47, 99, 4, 22, 90, 12, 19], [74, 51, 67, 72, 21, 9, 57, 50, 0, 43, 80, 91, 58, 46, 92, 98, 11, 4, 36, 31, 90, 90, 91, 52, 68, 63], [95, 76, 24, 52, 3, 71, 19, 75, 34, 92, 83, 15, 77, 12, 96, 58, 63, 68, 75, 9, 28, 44, 30, 94, 67, 49], [22, 93, 33, 77, 2, 9, 3, 3, 47, 56, 84, 70, 15, 81, 16, 49, 20, 95, 18, 22, 98, 3, 77, 27, 1, 13], [45, 63, 34, 0, 75, 45, 30, 23, 7, 7, 80, 62, 34, 11, 41, 16, 45, 6, 11, 21, 18, 55, 7, 24, 18, 70], [13, 7, 21, 85, 29, 53, 56, 83, 63, 89, 18, 67, 93, 73, 37, 3, 55, 65, 16, 72, 6, 80, 0, 39, 51, 24], [72, 23, 9, 56, 60, 88, 69, 6, 8, 92, 3, 44, 29, 5, 58, 58, 55, 24, 48, 57, 28, 69, 64, 72, 58, 98], [20, 56, 52, 74, 27, 95, 85, 20, 7, 52, 8, 93, 76, 53, 62, 54, 34, 25, 89, 38, 85, 29, 38, 18, 1, 28], [30, 97, 74, 11, 36, 92, 55, 74, 34, 29, 12, 40, 61, 69, 54, 72, 14, 64, 73, 75, 75, 4, 37, 47, 17, 29], [7, 18, 25, 6, 51, 63, 63, 53, 38, 96, 19, 56, 36, 35, 75, 99, 32, 28, 68, 14, 55, 9, 3, 19, 9, 59], [0, 98, 57, 98, 13, 25, 55, 56, 58, 37, 30, 90, 51, 71, 10, 36, 58, 94, 32, 80, 95, 44, 40, 82, 99, 6], [74, 28, 93, 37, 81, 54, 92, 89, 52, 96, 93, 8, 65, 82, 7, 14, 75, 0, 45, 59, 15, 17, 85, 87, 10, 52], [10, 74, 13, 23, 56, 25, 66, 59, 86, 39, 47, 72, 92, 28, 23, 75, 23, 18, 5, 20, 36, 52, 42, 56, 20, 7], [32, 37, 58, 20, 3, 33, 76, 92, 36, 73, 90, 53, 82, 78, 6, 66, 11, 33, 64, 68, 51, 76, 94, 94, 74, 88]]\n[[0.7236403458959406, 0.0956019387576047, 0.0025299248050427714, 0.8219024304497274, 0.43253754513562515, 0.8013795226500925], [0.1645225418615962, 0.17254764305062675, 0.915834884927677, 0.15659274788174238, 0.4408801726853846, 0.6712507398638423], [0.7220314060070252, 0.1109087497279424, 0.8673890374761482, 0.6019681601593759, 0.21136092547712715, 0.46410460250177055], [0.2051472970020488, 0.7021578939163269, 0.4920315519905448, 0.8786530949689468, 0.8406582658875078, 0.7656322995670249], [0.5314722945128192, 0.20582039242966288, 0.6649783801689887, 0.9122470167268962, 0.06046820688028054, 0.7640361944809368], [0.8531299103217095, 0.8837919293919477, 0.5584731093192602, 0.5488851769744959, 0.5426259488733682, 0.8101919492091457], [0.014691936047236509, 0.8299297933323541, 0.04420642840864686, 0.19514486051010316, 0.5605834763387445, 0.021425480951998255], [0.6251450063221531, 0.916013278510962, 0.9266733043623226, 0.4314909906070713, 0.5861250222822415, 0.6933275681854775], [0.613436662402963, 0.25971014970117345, 0.8516017571376222, 0.3946078968050868, 0.5030576607821642, 0.4947379037657953], [0.1831704150579163, 0.7027400367924451, 0.9687380255252486, 0.05161874874595729, 0.5662001008903554, 0.1163342848387866], [0.7871817922619593, 0.2744881375377821, 0.47927673745333677, 0.29916960674176196, 0.36165825794500894, 0.4473132902029585], [0.9460136327515588, 0.9784510345542305, 0.7292583652396575, 0.9967710630754123, 0.5222338378761943, 0.15774056366446398], [0.5663154377790205, 0.31992559317458047, 0.895903341426127, 0.10800834113175917, 0.7025174488794499, 0.09983260287294515], [0.2870859802344229, 0.6124244361792336, 0.03043370710190285, 0.4177754705816856, 0.41076530192454186, 0.059229404317664214], [0.453421549409623, 0.10006035499361832, 0.4729640823042591, 0.4187735846604017, 0.19252902582118436, 0.4571615927038022], [0.4717366823003555, 0.311470963714212, 0.7563429074261462, 0.9450429903711869, 0.23851560864324461, 0.4264206092799121], [0.14209483434392234, 0.9545183136517666, 0.02853067102355411, 0.8397788414889452, 0.28747164060068653, 0.5890799959267197], [0.7137144457967627, 0.7108041311984524, 0.3391605131543025, 0.18466650700703768, 0.07037926283668172, 0.1691030355977058], [0.4181167385409663, 0.5733773938988352, 0.9308794863064511, 0.955104551017489, 0.7472618752255964, 0.9106883383537705], [0.29209827546854006, 0.7950653331872178, 0.9314779081831699, 0.2137419943082265, 0.9590688802321072, 0.21779623076769017], [0.6414528631722118, 0.7772400748403205, 0.7240597746441493, 0.4846785371953165, 0.20903895145878393, 0.9928711008461597], [0.4987552039133927, 0.966261456826001, 0.6392910461562884, 0.3891694028095307, 0.14376415691424704, 0.5654942409405452], [0.39062876410463865, 0.4372793328535669, 0.9066881332880398, 0.928194141998039, 0.26891611788606773, 0.970014111003586], [0.05753018657343756, 0.5987554892139141, 0.6695393400712614, 0.4342378657370556, 0.5068004463455815, 0.28913437767829675], [0.31284712702847906, 0.6696586256781413, 0.6349611781499843, 0.11008282689008553, 0.9000387199581723, 0.5893732652223279], [0.38771861901614457, 0.9275236976874062, 0.1507893346167909, 0.2649576462980838, 0.8917999241041804, 0.7060665522096253]]\n[[0.7146175764456325, 0.31161087332596693, 0.799868898982844, 0.3303984762074823, 0.15755367025489198, 0.8822561515814714], [0.48324415449065805, 0.32294633607735035, 0.273076894762348, 0.46575965932905583, 0.35173647464295466, 0.0698782343365999], [0.05951092454514029, 0.9631544906381114, 0.14919875559361273, 0.9071033838543416, 0.9235221236014998, 0.15343960980130578], [0.37667471994346735, 0.3832592710693109, 0.1372971042292026, 0.5063394603470396, 0.3657347277059969, 0.21520394748123772], [0.5589413502705171, 0.9228726685280682, 0.9028006349689756, 0.7902921185261006, 0.09337560160131464, 0.8806823905125992], [0.19078196327854235, 0.9862705503057667, 0.00800242331367751, 0.7036641885324555, 0.7452071471082209, 0.85314563397203], [0.42059808696733225, 0.3678976649279547, 0.15153142787888962, 0.9831212856723789, 0.7218055186807681, 0.8943329971799489], [0.058672278269596756, 0.6364681756816949, 0.24610924719747507, 0.8429515887557353, 0.23639035927773622, 0.9193123017124043], [0.3667295853360063, 0.46010540148263646, 0.818107188288508, 0.027140526241385965, 0.4420026102807323, 0.3050634480740779], [0.9602073143407148, 0.5408373879572825, 0.4027285042008486, 0.854769594232319, 0.8977332204421882, 0.7804511190784789], [0.9554030213710992, 0.6286064807931032, 0.7899715293283952, 0.20778805629585584, 0.34452317136784105, 0.8373278109724016], [0.9511367017094053, 0.8108673965379353, 0.5917802839407773, 0.08638924272725734, 0.5614389008614823, 0.10285577516634681], [0.15293610366355237, 0.4726630546010566, 0.7151593451811216, 0.6787398883364194, 0.05726564336395312, 0.11175850750236216], [0.5284613368117816, 0.2171952870224766, 0.14730305464381344, 0.16327154211081985, 0.22473713798444594, 0.8780618686814565], [0.6229749344457463, 0.2450307938022901, 0.856716114606889, 0.5130970556519491, 0.09638792050417233, 0.5580480219996736], [0.02557257524793899, 0.16614696997999867, 0.9057474930205891, 0.9639638373151679, 0.8305505098688646, 0.13212730388642224], [0.9945224728362285, 0.601015567237635, 0.627777689771871, 0.062014306890884385, 0.5482657713187832, 0.050645865034282034], [0.222530564647055, 0.16270913407631815, 0.3463743065572499, 0.3642479732760492, 0.6787809827842912, 0.6698646733332234], [0.0514618465561959, 0.09146560753484756, 0.5663782403169225, 0.09809277695721963, 0.7435268283749681, 0.6941669527997202], [0.7220783710745816, 0.242189075941737, 0.19197963437514165, 0.2789768605860322, 0.1257100212184865, 0.25803379668907667], [0.5231678066470311, 0.4611093289035184, 0.8420569136872692, 0.9566490894261072, 0.07691192438283945, 0.37613366065780873], [0.010481514678729265, 0.5145103851754453, 0.9425491945781952, 0.24440293940943314, 0.2766636476384883, 0.9944680222564074], [0.7081598239606982, 0.3291415847107684, 0.7986116830970068, 0.32005951294163504, 0.988878016430301, 0.16702718654180948], [0.3281899603978248, 0.8371583970360936, 0.914781121298489, 0.9898376984561366, 0.2605393835200668, 0.7046307961318979], [0.6669241697435273, 0.7506837943872975, 0.3223310011040579, 0.8024412673323509, 0.47139557621217376, 0.34991596973647043], [0.6981065171211724, 0.7907802796637423, 0.05700463849852977, 0.29301210116680565, 0.3246921756526622, 0.9147896908728982]]\n0.5878728651551414\n97\n0.8066056621137515""", 
READER = (x for x in SAMPLES[0].split('\n')); 
sys.stdin.readline = lambda: next(READER)       # тупо шоп не переписывать

import sys
import ast
import numpy as np


def parse_array(s):
    return np.array(ast.literal_eval(s))

def read_array():
    return parse_array(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))


def update_glove_weights(x, w, d, alpha, max_x, learning_rate):
    """
    x - square integer matrix VocabSize x VocabSize - coocurrence matrix
    w - VocabSize x EmbSize - first word vectors
    d - VocabSize x EmbSize - second word vectors
    alpha - float - power in weight smoothing function f
    max_x - int - maximum coocurrence count in weight smoothing function f
    learning_rate - positive float - size of gradient step
    """
    fx = np.where(x <= max_x, (x / max_x) ** alpha, 1)
    y = np.log(1 + x)
    
    d_w = 2 * fx * (w @ d.T - y) @ d
    d_d = (2 * fx * (w @ d.T - y)).T @ w
    w -= learning_rate * d_w
    d -= learning_rate * d_d


x = read_array()
w = read_array()
d = read_array()
alpha = float(sys.stdin.readline().strip())
max_x = int(sys.stdin.readline().strip())
learning_rate = float(sys.stdin.readline().strip())

update_glove_weights(x, w, d, alpha, max_x, learning_rate)

# write_array(w) # [[37.894949622265095, 35.32608453541813, 39.58755103784916, 33.56742861491748, 31.889707572091282, 33.00715086688418], ...
# write_array(d)  # [[25.737584343831532, 31.976583863756073, 31.991796072794237, 30.397770296625676, 29.155071762578395, 28.039906267294363], ...

Вы обучили некоторую модель эмбеддингов и получили матрицу векторов слов.

Напишите функцию, которая для заданного слова получит список наиболее похожих на него слов вместе с оценкой близости. Результат должен быть отсортирован по убыванию оценки сходства (наиболее похожие вначале). Само слово тоже должно присутствовать в выдаче.

Для оценки сходства используйте отрицательное эвклидово расстояние $sim(a, b) = - \sqrt{\sum_i {(a_i - b_i) ^ 2}}$. Перед вычислением сходства нормируйте векторы по L2-норме (разделить вектор на евклидову норму этого же вектора).

Тесты (включая тест для примера) генерируются случайно.

Функция принимает на вход:

- embeddings - матрица размерности VocabSize x EmbSize - заранее выученная матрица векторов слов (эмбеддингов)
- query_word_id - целое неотрицательное число - идентификатор (номер) слова-запроса, для которого нужно найти наиболее похожие слова
- get_n - целое неотрицательное число - наибольшее количество похожих слов, которое нужно вернуть из функции

Функция должна возвращать список из не более чем get_n пар (номер слова, оценка сходства), отсортированных по убыванию оценки сходства со словом-запросом.

In [114]:
SAMPLES = """[[0.7299015792584768, 0.2915364327741303, 0.5307571134639943, 0.3101345732086396, 0.8327085262119636, 0.39018382511314353, 0.678094726221033, 0.12372148102696612, 0.5966533433209616], [0.5411155947267721, 0.046791742239819856, 0.5358832195593092, 0.09894162419462038, 0.6350557173679914, 0.15126161842015717, 0.11375720216711405, 0.46954553941325416, 0.8281402097264261], [0.5323869209381028, 0.2005012376766715, 0.5925043884236925, 0.4621530177251649, 0.3886830034303448, 0.6403738184472031, 0.23320289120963578, 0.43574647265888766, 0.5305633832484254]]\n0\n8""", 
READER = (x for x in SAMPLES[0].split('\n')); 
sys.stdin.readline = lambda: next(READER)       # тупо шоп не переписывать

import sys
import ast
import numpy as np


def parse_array(s):
    return np.array(ast.literal_eval(s))

def read_array():
    return parse_array(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))


def get_nearest(embeddings, query_word_id, get_n):
    """
    embeddings - VocabSize x EmbSize - word embeddings
    query_word_id - integer - id of query word to find most similar to
    get_n - integer - number of most similar words to retrieve

    returns list of `get_n` tuples (word_id, similarity) sorted by descending order of similarity value
    """

    def sim(a, b):
        return -np.linalg.norm(a - b)

    l2_normalized = embeddings / np.linalg.norm(embeddings, axis=1).reshape(-1, 1)
    # суровый нампай
    norms = np.vectorize(sim, signature='(d),(d)->()')(l2_normalized, l2_normalized[query_word_id])
    # суровый нампай
    res = np.vstack((np.argsort(-norms),  -np.sort(-norms))).T.tolist()

    return res[:get_n]
    
    # или тоже самое в более понятном виде
    return np.vstack((np.argsort(-(np.vectorize(lambda a, b: -np.linalg.norm(a - b), signature='(d),(d)->()')((embeddings / np.linalg.norm(embeddings, axis=1).reshape(-1, 1))[query_word_id], embeddings / np.linalg.norm(embeddings, axis=1).reshape(-1, 1)))),  -np.sort(-(np.vectorize(lambda a, b: -np.linalg.norm(a - b), signature='(d),(d)->()')((embeddings / np.linalg.norm(embeddings, axis=1).reshape(-1, 1))[query_word_id], embeddings / np.linalg.norm(embeddings, axis=1).reshape(-1, 1)))))).T.tolist()[:get_n]


embeddings = read_array()
query_word_id = int(sys.stdin.readline().strip())
get_n = int(sys.stdin.readline().strip())

result = get_nearest(embeddings, query_word_id, get_n)

write_array(np.array(result))

[[0.0, -0.0], [2.0, -0.5028921892757982], [1.0, -0.542231021984391]]
